In [1]:
import os
import glob
import torch
from tqdm import tqdm
import numpy as np

# Aus deinem Projekt:
from Helper.ml_models import TrainedModel, K_Fold_Dataset

# Basisverzeichnis
BASE_PATH = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation"

# Verzeichnis mit den K-Fold-Runs
K_FOLD_RUN_DIR = os.path.join(BASE_PATH, "K_Fold_Run")

# Ausgabe-Verzeichnis für die Confusion Matrices, z.B.:
CONF_MATRIX_BASE = os.path.join(BASE_PATH, "FINAL_DATEN", "confusion_matrices_k_fold")
os.makedirs(CONF_MATRIX_BASE, exist_ok=True)

# Cityscapes hat 20 Klassen
NUM_CLASSES = 20

print("K_FOLD_RUN_DIR:", K_FOLD_RUN_DIR)
print("CONF_MATRIX_BASE:", CONF_MATRIX_BASE)

/home/jan/anaconda3/envs/studi/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-03-21 15:59:08.596183: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-21 15:59:09.200194: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


K_FOLD_RUN_DIR: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/K_Fold_Run
CONF_MATRIX_BASE: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_k_fold


In [2]:
def compute_confusion_matrix(predicted, ground_truth, num_classes):
    """
    Berechnet die Confusion Matrix für ein einzelnes Bild.
    """
    mask = (ground_truth >= 0) & (ground_truth < num_classes)
    label = num_classes * ground_truth[mask] + predicted[mask]
    count = torch.bincount(label, minlength=num_classes**2)
    confusion_matrix = count.reshape(num_classes, num_classes)
    return confusion_matrix


In [3]:
# Wir gehen davon aus, dass die Folds von 0 bis 4 laufen.
# Für jeden Fold wird leave_out_fold = fold als Testset genutzt.
NUM_FOLDS = 5

for fold in range(NUM_FOLDS):
    print(f"\n=== Starte Auswertung für Fold {fold} ===")
    
    # Erstelle den K-Fold-Datensatz mit dem entsprechenden leave_out_fold
    k_fold_dataset = K_Fold_Dataset(
        image_dir='CityscapesDaten/images',
        annotation_dir='CityscapesDaten/semantic',
        k_fold_csv_dir='Daten/CityscapesDaten',
        leave_out_fold=fold
    )
    print(f"Test-Dataset Größe (Fold {fold}): {len(k_fold_dataset.test_dataset)}")
    
    # Finde alle Ordner in K_Fold_Run, die zu diesem Fold gehören,
    # also z.B. "deeplabv3_mobilenet_v3_large_k_fold_0"
    pattern = f"*_k_fold_{fold}"
    fold_folders = glob.glob(os.path.join(K_FOLD_RUN_DIR, pattern))
    
    # Erstelle das Ausgabe-Verzeichnis für diesen Fold
    fold_output_dir = os.path.join(CONF_MATRIX_BASE, f"Fold{fold}")
    os.makedirs(fold_output_dir, exist_ok=True)
    
    # Iteriere über alle gefundenen Modellordner
    for model_folder_path in fold_folders:
        base_name = os.path.basename(model_folder_path)
        # Beispiel: "deeplabv3_mobilenet_v3_large_k_fold_0"
        # Modellname wäre "deeplabv3_mobilenet_v3_large"
        # => wir entfernen "_k_fold_0" vom Ordnernamen
        model_name = base_name.rsplit(f"_k_fold_{fold}", 1)[0]
        
        print(f"\nLade Modell: {model_name}, Fold: {fold}")
        
        # Lade das Modell mithilfe von TrainedModel
        # Die Parameter (width, height) anpassen, wenn deine Modelle andere Input-Shapes erwarten
        # (bspw. 2048 x 1024 wie in deinem Beispiel).
        model = TrainedModel(
            model_name=model_name,
            width=2048,
            height=1024,
            weights_name=base_name,     # => "deeplabv3_mobilenet_v3_large_k_fold_0"
            folder_path=K_FOLD_RUN_DIR,
            start_epoch='latest'        # Lädt den neuesten Checkpoint
        )
        
        # Modell in den Evaluationsmodus setzen
        model.model.eval()
        
        # Initialisiere die Confusion Matrix für dieses Modell
        confusion_matrix_total = torch.zeros((NUM_CLASSES, NUM_CLASSES), dtype=torch.int64)
        
        # Test-Datensatz durchlaufen
        with torch.no_grad():
            for i in tqdm(range(len(k_fold_dataset.test_dataset)), desc=f"Evaluierung {model_name}"):
                image, annotation = k_fold_dataset.test_dataset[i]
                
                # Inferenz
                output = model.inference(image)
                predicted = output.argmax(1).squeeze(0)
                
                # Aktualisiere die Confusion Matrix
                cm = compute_confusion_matrix(predicted.cpu(), annotation.cpu(), NUM_CLASSES)
                confusion_matrix_total += cm
        
        # Speichere die Confusion Matrix
        cm_save_path = os.path.join(fold_output_dir, f"{model_name}_confusion_matrix.pt")
        torch.save(confusion_matrix_total, cm_save_path)
        print(f"Confusion Matrix gespeichert unter: {cm_save_path}")




=== Starte Auswertung für Fold 0 ===
Test-Dataset Größe (Fold 0): 695

Lade Modell: deeplabv3_mobilenet_v3_large, Fold: 0
Using CUDA GPU
Model loaded: deeplabv3_mobilenet_v3_large | Device: cuda 


Evaluierung deeplabv3_mobilenet_v3_large: 100%|██████████| 695/695 [00:55<00:00, 12.54it/s]


Confusion Matrix gespeichert unter: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_k_fold/Fold0/deeplabv3_mobilenet_v3_large_confusion_matrix.pt

Lade Modell: deeplabv3_resnet50, Fold: 0
Using CUDA GPU
Model loaded: deeplabv3_resnet50 | Device: cuda 


Evaluierung deeplabv3_resnet50: 100%|██████████| 695/695 [01:00<00:00, 11.53it/s]


Confusion Matrix gespeichert unter: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_k_fold/Fold0/deeplabv3_resnet50_confusion_matrix.pt

Lade Modell: deeplabv3_resnet101, Fold: 0
Using CUDA GPU
Model loaded: deeplabv3_resnet101 | Device: cuda 


...

Confusion Matrix gespeichert unter: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_k_fold/Fold4/deeplabv3_mobilenet_v3_large_confusion_matrix.pt

Lade Modell: fcn_resnet50, Fold: 4
Using CUDA GPU
Model loaded: fcn_resnet50 | Device: cuda 


Evaluierung fcn_resnet50: 100%|██████████| 695/695 [00:58<00:00, 11.82it/s]


Confusion Matrix gespeichert unter: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_k_fold/Fold4/fcn_resnet50_confusion_matrix.pt

Lade Modell: lraspp_mobilenet_v3_large, Fold: 4
Using CUDA GPU
Model loaded: lraspp_mobilenet_v3_large | Device: cuda 


Evaluierung lraspp_mobilenet_v3_large: 100%|██████████| 695/695 [00:50<00:00, 13.81it/s]

Confusion Matrix gespeichert unter: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/confusion_matrices_k_fold/Fold4/lraspp_mobilenet_v3_large_confusion_matrix.pt
